# Import Fire Data, the loop through and join weather data for each year

## Dataset creation steps

1. **firedf**: 
    * import fire dataset
    * create composite date + refkey column for joining weatherdata
    * drop clearly unnecessary columns
    `['DISCOVERY_DOY','LATITUDE','LONGITUDE','ref_key']`
2. **weather_df**: loop through weather datasets for each year
    * drop clearly unnecessary columns:
    `['stn','wban','year','mo','da','count_temp','count_dewp','count_slp','count_stp',
       'count_visib','count_wdsp','flag_max','flag_min','flag_prcp']`
    * merge to fire data subset for each year
3. Remove remaining extraneous columns
    `['FORMATTED_DATE','weather_obs_key','date_ref_key','stp','sndp','ref_key','STATE']`
4. Create `month` column and drop `formatted_date` for analysis
5. Export dataset entitled: `pre_cleaning_dataset.csv`

In [1]:
#import packages
import pandas as pd
import calendar
from tqdm import tqdm_notebook as tqdm

In [2]:
#import fire dataset
firedf = pd.read_csv('full_fire_output_allstations_345.csv')
#create date reference key column for merging
firedf['date_ref_key'] = firedf['FORMATTED_DATE'].astype('str') + firedf['weather_obs_key'].astype('str')
#drop unnecessary firedf columns
firedf.drop(columns=['DISCOVERY_DOY','LATITUDE','LONGITUDE','ref_key'], inplace=True)

In [3]:
#declare final dataframe
final_df = pd.DataFrame()
#loop through weather datasets for each year and join to fire data
for year in tqdm(range(1992, 2016)):
    #get weather data csv
    weather_df = pd.read_csv('weatherdata/US_weather_{}.csv'.format(year), low_memory=False)
    #create date column yyyy-mm-dd
    weather_df['formatted_date'] = pd.to_datetime(weather_df['year']*10000+weather_df['mo']*100+weather_df['da'],
                                                    format='%Y%m%d')
    #create refkey column
    weather_df['ref_key'] = weather_df['stn'].astype('str') + weather_df['wban'].astype('str')
    #create daterefkey column for merging
    weather_df['date_ref_key'] = weather_df['formatted_date'].astype('str') + weather_df['ref_key']
    #drop unnecessary weather_df columns
    weather_df.drop(columns=['stn','wban','year','mo','da','count_temp','count_dewp','count_slp','count_stp',
                             'count_visib','count_wdsp','flag_max','flag_min','flag_prcp'], inplace=True)
    #segment firedf for specified year
    firedf_year = firedf[firedf['FIRE_YEAR'] == year]
    #mergedataframs per yer segment
    firedf_year = pd.merge(firedf_year, weather_df, how='inner', left_on='date_ref_key',right_on='date_ref_key')
    final_df = final_df.append(firedf_year)

In [4]:
#drop reference columns, redundant columns, and columns with not enough data
final_df.drop(columns=['FORMATTED_DATE','weather_obs_key','date_ref_key',
                       'stp','sndp','ref_key','STATE'], inplace=True)

In [5]:
#create `month` variable and drop `formatted_date`
#convert date column to datetime
final_df['formatted_date'] = pd.to_datetime(final_df['formatted_date'])
#create month column
final_df['month'] = final_df['formatted_date'].dt.month
#drop date column
final_df.drop(columns='formatted_date', inplace=True)
#convert integer values to month name
final_df['month'] = final_df['month'].apply(lambda x: calendar.month_abbr[x])

In [6]:
#export final_df to csv
final_df.to_csv('pre_cleaning_dataset.csv', index=False)